# 上下文无关文法分析

**分析器**根据文法产生式处理输入的句子，并建立一个或多个符合文法的组成结构。文法是一个格式良好的声明规范，它实际上只是一个字符串，而不是程序。分析器是文法的解释程序，它搜索符合文法的所有树的空间找出一棵边缘有所需句子的树。

在本节中，我们将看到两个简单的分析算法，一种自上而下的方法成为递归下降分析，一种自下而上的方法成为移进-规约分析。我们也将看到一些更复杂的算法，一种带自下而上过滤的自上而下的方法称为左角落分析，一种动态规划技术称为图表分析。

## 递归下降分析

一种最简单的分析器将一个文法作为如何将一个高层次的目标分解成几个低层次的子目标的规范来解释。顶层的目标是找到一个 S，S -> NP VP 产生式允许分析器替换这个目标为两个子目标：找到一个 NP，然后找到一个 VP。每个这些子目标都可以再次被子目标的子目标替代，使用左侧有 NP 和 VP 的产生式。递归下降分析器在上述过程中建立分析树，带着最初的目标创建根节点 S，然后随着使用文法的产生式递归扩展不断向下延伸分析树。

![rdparser1-6.png](resources/rdparser1-6.png)

在这个过程中，分析器往往被迫在多种可能的产生式种选择。例如：从第 3 步到第 4 步，它试图找到左侧有 N 的产生式，第一个是 N -> man，当这不起作用时就回溯，按顺序尝试其他左侧有 N 的产生式，知道它得到 N -> dog，与输入句子的下一个词相匹配。一段时间后，如第 5 步所示，它发现了一个完整的分析树，这是一个涵盖了整个句子的树，没有任何悬着的边。发现了分析树后我们可以让分析器寻找其他额外的分析树，它会再次回溯和探索选择其他产生式，以免遗漏任何一个产生分析树的情况。

NLTK 提供了一个递归下降分析器 [nltk.RecursiveDescentParser](http://www.nltk.org/_modules/nltk/parse/recursivedescent.html#RecursiveDescentParser)，它可以接受一个可选的参数 trace，如果 trace 大于零，分析器将报告它解析文本的步骤：

In [1]:
import nltk

grammar1 = nltk.CFG.fromstring("""
    S -> NP VP
    VP -> V NP | V NP PP
    PP -> P NP
    V -> "saw" | "ate" | "walked"
    NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
    Det -> "a" | "an" | "the" | "my"
    N -> "man" | "dog" | "cat" | "telescope" | "park"
    P -> "in" | "on" | "by" | "with"
""")

rd_parser = nltk.RecursiveDescentParser(grammar1)
sent = 'Mary saw a dog'.split()
for tree in rd_parser.parse(sent):
    print(tree)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


递归下降分析有三个主要的缺点：首先，左递归产生式，如：NP -> NP PP，会进入死循环。第二，分析器浪费了很多时间处理不符合输入句子的词和结构。第三，回溯过程中可能会丢弃分析过的成分，它们将需要在之后再次重建。

递归下降分析时一个**自上而下分析**，自上而下分析器在检查输入之前先使用文法预测输入将是什么。然而，由于输入对分析器一直是可用的，从一开始就考虑输入的句子会是更明智的做法，这种做法被称为**自下而上分析**。

## 移进-规约分析

一种简单的自下而上分析器是移进-规约分析器。与所有自下而上的分析器一样，移进-规约分析器尝试找到对应文法产生式**右侧**的词和短语的序列，用左侧的替换它们，直到整个句子规约为一个 S。

移进-规约分析器反复将下一个输入词推到堆栈，这是**移进**操作。如果堆栈上的前 n 项匹配一些产生式右侧的 n 个项目，那么就把它们弹出栈，并把产生式左边的项目压入栈，这种替换前 n 项为一项的操作就是**规约**操作。此操作只适用于堆栈的顶部，规约栈中的项目必须在后面的项目被压入栈之前做。当所有的输入都使用过，堆栈中只剩一个项目，也就是一棵分析树作为它的根节点 S 时，分析器完成。

我们可以使用图形化示范 nltk.app.srparser() 看到移进-规约分析算法执行步骤：

![srparser1-6.png](resources/srparser1-6.png)

NLTK 中提供了 [nltk.ShiftReduceParser](http://www.nltk.org/_modules/nltk/parse/shiftreduce.html#ShiftReduceParser)，移进-规约分析器的一个简单实现。这个分析器不执行任何回溯，所以它不能保证一定能找到一个文本的解析，即使确实存在这样一个解析。此外，它最多只会找到一个解析，即使有多个解析存在。

In [2]:
sr_parser = nltk.ShiftReduceParser(grammar1)
sent = 'Mary saw a dog'.split()
for tree in sr_parser.parse(sent):
    print(tree)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


即使输入的句子是符合语法的，移进-归于分析器也有可能会到达一个死胡同，而不能找到任何解析，问题出现的原因是较早前做出的选择不能被分析器撤销。

在分析的过程中，分析器通常面临两种选择：当有多种规约可能时选择哪个规约（规约-规约冲突）；当移进和规约都可以时选择哪个动作。移进-规约分析器可以改进执行策略来解决这些冲突。例如：它可以通过只有在不能规约时才移进，解决移进-规约冲突；它可以通过优先执行从堆栈中移除更多项的规约操作，解决规约-规约冲突。

移进-规约分析器相比递归下降分析器的好处是，它们只建立与输入中的词对应的结构，而且每个结构只建立一次。例如：NP(Det(the), N(man)) 只建立和压入栈一次，不管以后 VP -> V NP PP 规约或者 NP -> NP PP 规约会不会用到。

## 左角落分析器

递归下降分析器的问题之一是当它遇到一个左递归产生式时，会进入无限循环，这是因为它盲目应用文法产生式而不考虑实际输入的句子，可以使用左角落分析器解决这一问题。

**左角落分析器**是一个带自下而上过滤的自上而下分析器。在开始工作之前，左角落分析器预处理上下文无关文法建立一个表，其中每行包含两个单元，第一个存放非终结符（如 S、NP、VP 等），第二个存放那个非终结符可能的左角落集合。用 grammar1 的文法演示如下：

| 类型 | 左角落（非终结符） |
|------|--------------------|
| S    | NP                 |
| NP   | Det                |
| VP   | V                  |
| PP   | P                  |

分析器每次考虑产生式时，它会检查下一个输入词是否与左角落表格中至少一种非终结符的类别相容。例如分析 John saw Mary 这个句子，首先是 S -> NP VP，然后在替换 NP 时，我们发现 NP 的左角落是 Det，它和 John 不相容，因此可以掠过 NP -> Det N 和 NP -> Det N PP 这两条规则，直接与 NP -> 'John' | 'Mary' | 'Bob' 进行比较。

## 符合语法规则的子串表

上面讨论的三种简单的分析器在完整性和效率上都有限制，为了弥补这些，我们可以使用动态规划算法解决。动态规划算法可以存储计算的中间结果，并在适当的时候重用它们，能显著提高效率。这种技术应用到句法分析，我们可以存储分析任务的部分解决方案，然后在必要的时候查找它们，直到达到最终解决方案。这种分析方法叫做**图表分析**，存储中间结果的表格被称为**符合语法规则的子串表（well-formed substring table，WFST）。

我们将句子用如下的结构表示，那么 WFST[i][j] 表示的是从第 i 个节点到第 j 个节点的子串对应的结构类型。例如：shot 是从节点 1 到节点 2 的子串，且文法中有产生式 V -> 'short'，因此我们将 WSFT[1][2] 设置为 V。

![chart_positions1.png](resources/chart_positions1.png)

In [3]:
groucho_grammar = nltk.CFG.fromstring("""
    S -> NP VP
    PP -> P NP
    NP -> Det N | Det N PP | 'I'
    VP -> V NP | VP PP
    Det -> 'an' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
""")

text = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
print(groucho_grammar.productions(rhs=text[1]))

[V -> 'shot']


这里我们定义 init_wfst 方法来初始化 WFST 表格，定义单个单词所属的类别。其中用到了 grammar 对象的 [productions](https://www.nltk.org/_modules/nltk/grammar.html#CFG.productions) 方法来过滤符合条件的产生式，它可以接收 rhs 参数来匹配产生式右侧的第一个符号，也可以接收 lhs 参数匹配产生式左侧的符号。

In [4]:
def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens + 1)] for j in range(numtokens + 1)]
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        wfst[i][i + 1] = productions[0].lhs()
    return wfst

def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2, numtokens + 1):
        for start in range(numtokens + 1 - span):
            end = start + span
            for mid in range(start + 1, end):
                nt1, nt2 = wfst[start][mid], wfst[mid][end]
                if nt1 and nt2 and (nt1, nt2) in index:
                    wfst[start][end] = index[(nt1, nt2)]
                    if trace:
                        print("[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" % 
                               (start, nt1, mid, nt2, end, start, index[(nt1,nt2)], end))
    return wfst

def display(wfst, tokens):
    print('\nWFST ' + ' '.join(('%-4d' % i) for i in range(1, len(wfst))))
    for i in range(len(wfst) - 1):
        print("%d   " % i, end=" ")
        for j in range(1, len(wfst)):
            print("%-4s" % (wfst[i][j] or '.'), end=" ")
        print()

tokens = "I shot an elephant in my pajamas".split()
wfst0 = init_wfst(tokens, groucho_grammar)
display(wfst0, tokens)


WFST 1    2    3    4    5    6    7   
0    NP   .    .    .    .    .    .    
1    .    V    .    .    .    .    .    
2    .    .    Det  .    .    .    .    
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    .    
5    .    .    .    .    .    Det  .    
6    .    .    .    .    .    .    N    


应用动态规划的思想，对于单词 an 我们有 Det 位于（2，3）单元格，对于单词 elephant 我们有 N 位于（3，4）单元格，那么对于 an elephant 我们应该在（2，4）单元格中放入 NP，因为文法中有 NP -> Det N 的产生式。更一般的，我们可以在（i，j）输入 A，如果有一个产生式 A -> B C，并且我们在（i，k）中找到非终结符 B，在（k，j）中找到非终结符 C。

逐步增加 i 和 j 之间的步长，直到 i 和 j 分别代表整个字符串的开头和结尾，我们就填写完了整个 WFST 表格，具体代码可以参考 complete_wsft 方法：

In [5]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar)
display(wfst1, tokens)


WFST 1    2    3    4    5    6    7   
0    NP   .    .    S    .    .    S    
1    .    V    .    VP   .    .    VP   
2    .    .    Det  NP   .    .    .    
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    PP   
5    .    .    .    .    .    Det  NP   
6    .    .    .    .    .    .    N    


通过调用函数 complete_wfst() 时设置 trace 为 True，我们可以看到 WFST 创建过程中的跟踪输出：

In [6]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar, trace=True)

[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[1]  VP [4]  PP [7] ==> [1]  VP [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]


最终，只要我们在（0，7）单元格中可以构建 S 节点，就表明我们已经为整个输入字符串找到了一个解析，解析图如下所示：

![chart_positions2.png](resources/chart_positions2.png)

WFST 有几个缺点。首先，WFST 本身不是一个分析树，所以该技术严格地说是认识到一个句子被一个文法承认，而不是分析它。其次，它要求每个非词汇文法产生式是二元的，不过我们可以将任意的 CFG 转换为这种形式。第三，作为一个自下而上的方法，它潜在的存在浪费，它会在不符合文法的地方提出成分。

最后，WFST 不能表示句子中的结构歧义，如果一个子串有多种解析方式，前面的都会被最后一种覆盖。